In [2]:
import psycopg2
import pandas as pd


In [3]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'user': 'auser',
    'password': 'Practica1_g4',
    'dbname': 'postgres'
}
DB_ANOMALIAS = 'postgres'
CSV_PATH = ["C:\\Users\\ADMIN\\Documents\\Practica1-IDN\\IN_P1_G4\\dataset\\population_data.csv"]
BATCH_SIZE = 10
MAX_WORKERS = 10

In [4]:
def verificar_conexion():
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT 1;")
                _ = cur.fetchone()
        print("✅ Conexión exitosa a PostgreSQL.")
    except Exception as e:
        print(f"❌ Error al conectar: {e}")

In [5]:
def crear_tabla(column_names, tabla='population'):
    columns_str = ', '.join([f'"{col}" VARCHAR(100)' for col in column_names])
    sql = f"CREATE TABLE IF NOT EXISTS {tabla} (id SERIAL PRIMARY KEY,{columns_str});"
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                cur.execute(sql)
                conn.commit()
        print("✅ Tabla creada.")
    except Exception as e:
        print(f"❌ Error al crear la tabla: {e}")

In [6]:
def consulta_a_dataframe(query, params=None):
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            df = pd.read_sql_query(query, conn, params=params)
        print("✅ Consulta ejecutada con éxito.")
        return df
    except Exception as e:
        print(f"❌ Error al ejecutar consulta: {e}")
        return pd.DataFrame()  # retorna vacío si falla

In [7]:
def insertar_registros(data, tabla='population'):
    try:
        # Columnas entre comillas dobles si tienen espacios o mayúsculas
        columnas = ', '.join([f'"{col}"' for col in data.columns])
        placeholders = ', '.join(['%s'] * len(data.columns))
        sql = f"INSERT INTO {tabla} ({columnas}) VALUES ({placeholders});"
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                for row in data.itertuples(index=False, name=None):
                    cur.execute(sql, row)
            conn.commit()

        print(f"✅ Insertados {len(data)} registros.")

    except Exception as e:
        print(f"❌ Error al insertar dato: {e}")

In [8]:
def main():
    try:
        verificar_conexion()

        print("⏳ Leyendo archivo CSV...")
        for csv_path in CSV_PATH:
            df = pd.read_csv(csv_path, delimiter=',', encoding='utf-8')
            column_names = df.columns.tolist()
            crear_tabla(column_names)
            insertar_registros(df)
        print("⏳ Insertando registros en la base de datos...")
    except Exception as e:
        print(f"❌ Error en el proceso principal: {e}")

if __name__ == '__main__':
    main()

✅ Conexión exitosa a PostgreSQL.
⏳ Leyendo archivo CSV...
✅ Tabla creada.
✅ Insertados 264 registros.
⏳ Insertando registros en la base de datos...


# Generación de los tres DataFrames  principales en función a cada una de las fuentes de datos

In [9]:
## pueden crear varios df en base a la consulta que deseen hacer por ejemplo: df1
df1 = consulta_a_dataframe("SELECT * FROM population LIMIT 5;")
df1.drop(columns=['Unnamed: 62'], inplace=True)
df1.head()

✅ Consulta ejecutada con éxito.


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5384\1920335490.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)


,id,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,...,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0
1,2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,...,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0
2,3,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,...,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0
3,4,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,...,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
4,5,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,...,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0


In [10]:
df2 = consulta_a_dataframe("SELECT * FROM population LIMIT 15;")
df2.head(10)

✅ Consulta ejecutada con éxito.


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5384\1920335490.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)


,id,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,1,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,NaN
1,2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,NaN
2,3,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0,NaN
3,4,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,NaN
4,5,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0,NaN
5,6,Arab World,ARB,"Population, total",SP.POP.TOTL,92490932.0,95044497.0,97682294.0,100411076.0,103239902.0,...,348145094.0,356508908.0,364895878.0,373306993.0,381702086.0,390043028.0,398304960.0,406452690.0,414491886.0,NaN
6,7,United Arab Emirates,ARE,"Population, total",SP.POP.TOTL,92634.0,101078.0,112472.0,125566.0,138529.0,...,7666393.0,8270684.0,8672475.0,8900453.0,9006263.0,9070867.0,9154302.0,9269612.0,9400145.0,NaN
7,8,Argentina,ARG,"Population, total",SP.POP.TOTL,20619075.0,20953077.0,21287682.0,21621840.0,21953929.0,...,40799407.0,41223889.0,41656879.0,42096739.0,42539925.0,42981515.0,43417765.0,43847430.0,44271041.0,NaN
8,9,Armenia,ARM,"Population, total",SP.POP.TOTL,1874120.0,1941491.0,2009526.0,2077575.0,2144998.0,...,2888584.0,2877311.0,2875581.0,2881922.0,2893509.0,2906220.0,2916950.0,2924816.0,2930450.0,NaN
9,10,American Samoa,ASM,"Population, total",SP.POP.TOTL,20013.0,20486.0,21117.0,21882.0,22698.0,...,56227.0,55637.0,55320.0,55230.0,55307.0,55437.0,55537.0,55599.0,55641.0,NaN


#### Genera un resumen estadístico de los datos de las columnas numéricas y distribución percentiles.

In [11]:
df1.describe()

,id
count,5.000000
mean,3.000000
std,1.581139
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [12]:
df2.describe()

,id
count,15.000000
mean,8.000000
std,4.472136
min,1.000000
25%,4.500000
50%,8.000000
75%,11.500000
max,15.000000


### Muestra una tupla con la cantidad de filas y columnas

In [13]:
df1.shape

(5, 63)

In [14]:
df2.shape

(15, 64)

### Elimina filas duplicadas de los DataFrame

In [15]:
df1 = df1.drop_duplicates()


In [16]:
df2 = df2.drop_duplicates()


### Revisa y detecta valores nulos en las columnas de los DataFrame

In [17]:
df1.isnull().sum()

id                0
Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
                 ..
2013              0
2014              0
2015              0
2016              0
2017              0
Length: 63, dtype: int64

In [18]:
df2.isnull().sum()


id                0
Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
                 ..
2014              0
2015              0
2016              0
2017              0
Unnamed: 62       0
Length: 64, dtype: int64

### Revisa y detecta valores nulos en las filas de los DataFrame

In [19]:
df1.isnull().sum(axis=1)

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [20]:
df2.isnull().sum(axis=1)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

### Validación de cantidad de valores únicos por columnas de los DataFrame

In [21]:
df1.nunique()

id                5
Country Name      5
Country Code      5
Indicator Name    1
Indicator Code    1
                 ..
2013              5
2014              5
2015              5
2016              5
2017              5
Length: 63, dtype: int64

In [22]:
df2.nunique()


id                15
Country Name      15
Country Code      15
Indicator Name     1
Indicator Code     1
                  ..
2014              15
2015              15
2016              15
2017              15
Unnamed: 62        1
Length: 64, dtype: int64

### Validación de cantidad de valores únicos por filas de los DataFrame

In [23]:
df1.nunique(axis=1)

0    63
1    63
2    63
3    63
4    63
dtype: int64

In [24]:
df2.nunique(axis=1)


0     64
1     64
2     64
3     64
4     64
5     64
6     64
7     64
8     64
9     64
10    64
11    64
12    64
13    64
14    64
dtype: int64